In [3]:
import numpy as np
import scipy.sparse as sparse
from scipy.sparse.linalg import expm, eigs
from qiskit.opflow import PrimitiveOp, PauliTrotterEvolution
from qiskit.quantum_info import SparsePauliOp, Operator
from qiskit import Aer, execute

# All matrix functions operate with Scipy spare csc as input/return type

def char_to_pauli(char):
    """Returns the Pauli matrix for a given character in I, X, Y, Z"""
    char = char.upper() # convert lower to upper case
    if char == 'I':
        return np.matrix([[1,0],[0,1]])
    if char == 'X':
        return np.matrix([[0,1],[1,0]])
    if char == 'Y':
        return np.matrix([[0,-1j],[1j,0]])
    if char == 'Z':
        return np.matrix([[1,0],[0,-1]])
    raise Exception("Character '" + char + "' does not correspond to Pauli matrix")

def string_to_operator(string):
    """Calculates the operator (Kronecker product) from a Pauli string"""
    op = sparse.csc_matrix(1, dtype=np.complex128)
    for c in string:
        op = sparse.kron(op, char_to_pauli(c))
    return op

def pauli_unitary(string, coefficient):
    """Calculates the unitary matrix exp(-icH) for a given Pauli string H with coefficient c"""
    return expm(-1j * coefficient * string_to_operator(string))

def pauli_circuit(string, coefficient):
    """Quantum circuit for the given Pauli string with coefficient"""
    op = PrimitiveOp(SparsePauliOp(string, coefficient))
    unitary = PauliTrotterEvolution().convert(op.exp_i())
    return unitary.to_circuit().decompose()

def circuit_unitary(circuit, simulate = False):
    """Return unitary calculated from Quantum circuit
       if parameter simulate is True: simulate with Aer unitary backend
       default: obtain unitary from qiskit.Operator class
    """
    if simulate:
        backend = Aer.get_backend('unitary_simulator')
        job = execute(circuit, backend)
        result = job.result()
        u = result.get_unitary(circuit)
    else:
        u = Operator(circuit).data
    return sparse.csc_matrix(u, dtype=np.complex128)

def error(a, b):
    """Error between two equal sized unitaries a, b (sparse csc) defined
    as largest magnitude eigenvalue of a-b
    """
    ev = eigs(a-b, 1, which='LM', return_eigenvectors=False)
    return np.abs(ev[0])

class PauliHamiltonian:
    """Class for Hamiltonian built from Pauli strings
    Attributes:
        operators: list of Pauli strings, in order of coefficient size
        coefficients: dictionary {'Pauli String': coefficient}
        qubits: number of qbits 
    """
    operators = []
    coefficients = {}
    qubits = 0
    
    __matrix = None
    __unitary = None
    #__gate_unitaries = {}
    
    def matrix(self):
        """Hamiltonian in matrix form"""
        if self.__matrix is None: # matrix form has not been calculated yet
            n = self.dim()
            self.__matrix = sparse.csc_matrix((n, n), dtype=np.complex128)
            for o in self.operators:
                self.__matrix += self.coefficients[o] * string_to_operator(o)
        return self.__matrix
    
    def unitary(self):
        """exp(-iH) in matrix form"""
        if self.__unitary is None: # unitary has not been calculated yet
            self.__unitary = expm(-1j * self.matrix())
        return self.__unitary
    
    def dim(self):
        """Matrix dimension 2^n with n qubits"""
        return 2**self.qubits
    
def hamiltonian_from_file(filename):
    """Read Hamiltonian from file and return PauliHamiltonian object.
    File format: 'coefficient Pauli-string' per line, separated by space"""
    try:
        file = open(filename) # read file
    except:
        print("An exception occurred trying to read the file: " + filename) 
    lines = file.read().split('\n')
    h = PauliHamiltonian() # create object to return
    for l in lines:
        if len(l): # exclude empty lines
            [c,o] = l.split() # split in coefficient c and operator string o
            if h.qubits:
                if len(o) != h.qubits: # Make sure all operators have the same dimension
                    raise Exception("Operator string '" + o + "' does not match dimension (" + h.qubits + ")")
            else:
                h.qubits = len(o) # set no. qubits to length of operator string
            h.operators.append(o) # add operator to list of operators of object h
            h.coefficients[o] = float(c) # add to the dictionary of coefficients for h
    return h


h = hamiltonian_from_file("hamiltonian_ordered.txt")

In [ ]:
# max error of all Pauli unitaries:
m = 0
for o in h.operators:
    c = h.coefficients[o]
    e = error(pauli_unitary(o,c), circuit_unitary(pauli_circuit(o,c)))
    if e > m:
        m = e
print(m)

/ssd/andgros/anaconda3/lib/python3.9/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:144: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  warn('spsolve requires A be CSC or CSR matrix format',
/ssd/andgros/anaconda3/lib/python3.9/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:215: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  warn('spsolve is more efficient when sparse b '
